# **Conformal LSTM for Time-Series Anomaly Detection**
---
    Course - B9DA109: Machine Learning and Pattern Recognition
---
    Name - Anish Rao
    Student No - 20066423
---
##### Dataset - [Numenta Anomaly Benchmark dataset](https://github.com/numenta/NAB)
---
###### Colab link - [Google Colab](https://colab.research.google.com/drive/1E4ATUk2mBCizDGUXYAOVS_rvqgPPxbFe?usp=sharing)
---

# Imports

In [1]:
import Pkg
packages = ["BSON", "CSV", "DataFrames", "DelimitedFiles",
"Flux", "JSON", "Statistics"]
for pkg in packages
     Pkg.add(pkg)
end
# Pkg.update()

using BSON: @save
using CSV
using DataFrames
using Dates
using DelimitedFiles
using Flux
using JSON
using Random
using Statistics

Random.seed!(86)

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
Precompiling packages...
┌ Warning: attempting to remove probably stale pidfile
│   path = "/root/.julia/compiled/v1.10/ReactantNNlibExt/aSIzU_MJWkB.ji.pidfile"
└ @ FileWatching.Pidfile /usr/local/share/julia/stdlib/v1.10/FileWatching/src/pidfile.jl:244
  59973.2 ms  ✓ Reactant → ReactantNNlibExt
   6915.7 ms  ✓ Plots → IJuliaExt
   9923.6 ms  ✓ Plots → GeometryBasicsExt
   6982.7 ms  ✓ Plots → FileIOExt
  21215.9 ms  ✓ Lux → LuxReactantExt
 391279.9 ms  ✓ Makie
 394523.0 ms  ✓ Reactant → ReactantCUDAExt
  7 dependencies successfully precompiled in 398 seconds. 507 already precompiled.
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Change

TaskLocalRNG()

# Unsupervised

### Parameter Loading

In [20]:
function load_params(json_path::String)::Dict{String, Dict{String, Float64}}
    if isfile(json_path)
        return JSON.parsefile(json_path)
    else
        error("params_unsupervised.json not found at $json_path")
    end
end

load_params (generic function with 1 method)

### Data Loading

In [21]:
function load_csv_data(filepath::String)
    df = CSV.read(filepath, DataFrame)
    timestamps = DateTime.(df.timestamp, dateformat"yyyy-MM-dd HH:MM:SS")
    values = Float32.(df.value)
    return timestamps, values
end

load_csv_data (generic function with 1 method)

### Data Normalisation and Splitting

In [22]:
function split_and_normalize(values::Vector{Float32}, prob_ratio::Float64, calib_ratio::Float64)
    n = length(values)
    prob_count = Int(floor(prob_ratio * n))
    calib_count = Int(floor(calib_ratio * prob_count))
    train_count = prob_count - calib_count

    probation_vals = values[1:prob_count]
    mean_prob = mean(probation_vals)
    std_prob = std(probation_vals)
    normal_values = (values .- mean_prob) ./ std_prob

    train_data = normal_values[1:train_count]
    calib_data = normal_values[train_count+1 : prob_count]
    test_data  = normal_values[prob_count+1 : end]

    return (;train_data, calib_data, test_data)
end

split_and_normalize (generic function with 2 methods)

### Data Sequencing

In [23]:
function create_sequences(values::Vector{Float32}, window_size::Int)
    n_seq = length(values) - window_size
    x = Array{Float32}(undef, window_size, 1, n_seq)
    y = Array{Float32}(undef, 1, n_seq)

    for i in 1:n_seq
        x[:, 1, i] = values[i : i + window_size - 1]
        y[1, i] = values[i + window_size]
    end

    return x, y
end

create_sequences (generic function with 2 methods)

### Model Training

In [24]:
function build_lstm_model(window_size::Int, dropout::Float64)
    return Chain(
        LSTM(window_size => 64),
        Dropout(dropout),
        Dense(64 => 1)
    )
end

function train_model(x_train::Array{Float32, 3}, y_train::Array{Float32,2};
    window_size::Int, epochs::Int, lr::Float64, dropout::Float64)

    model = build_lstm_model(window_size, dropout)
    opt_state = Flux.setup(ADAMW(lr), model)

    for epoch in 1:epochs
        Flux.reset!(model)

        loss, grads = Flux.withgradient(model) do m
            y_pred = m(x_train)
            y_pred = dropdims(y_pred, dims=2)
            Flux.Losses.mse(y_pred, y_train)
        end

        Flux.Optimise.update!(opt_state, model, grads[1])
        println("Epoch $epoch -- Loss = $(round(loss, digits=4))")
    end

    return model
end

train_model (generic function with 1 method)

### Conformal Classification

In [25]:
function conformal_classification(model, x_calib, y_calib; confidence::Float64)
    residuals = Float32[]

    for i in 1:size(x_calib, 3)
        x = x_calib[:, :, i]
        Flux.reset!(model)
        y_pred = model(x)
        push!(residuals, abs(y_pred[1] - y_calib[1, i]))
    end

    if any(isnan, residuals)
        return NaN, residuals
    end

    threshold = quantile(residuals, confidence)

    println("Threshold = ", round(threshold, digits=4))
    return threshold, residuals
end

conformal_classification (generic function with 1 method)

### Evaluation

In [26]:
function evaluate_model(model, x_eval, y_eval, threshold)
    residuals = Float32[]
    predictions = Float32[]
    anomaly_flags = Int[]

    for i in 1:size(x_eval, 3)
        x = x_eval[:, :, i]
        Flux.reset!(model)
        y_pred = model(x)[1]
        res = abs(y_pred - y_eval[1, i])

        push!(residuals, res)
        push!(predictions, y_pred)
        push!(anomaly_flags, res > threshold ? 1 : 0)
    end

    return (;predictions, residuals, anomaly_flags)
end

function load_true_labels(label_path::String, file_key::String,
    eval_timestamps::Vector{DateTime})

    label_data = JSON.parsefile(label_path)
    windows = label_data[file_key]

    datetime_format = dateformat"yyyy-MM-dd HH:MM:SS.ssss"
    anomaly_windows = [(DateTime(w[1], datetime_format),
    DateTime(w[2], datetime_format)) for w in windows]

    true_flags = Int[]
    for ts in eval_timestamps
        is_anomaly = any(r -> ts ≥ r[1] && ts ≤ r[2], anomaly_windows)
        push!(true_flags, is_anomaly ? 1 : 0)
    end

    return true_flags
end

function compute_metrics(filename::String, pred_flags::Vector{Int}, true_flags::Vector{Int})
    TP = sum((pred_flags .== 1) .& (true_flags .== 1))
    FP = sum((pred_flags .== 1) .& (true_flags .== 0))
    FN = sum((pred_flags .== 0) .& (true_flags .== 1))

    precision = TP + FP == 0 ? 0.0 : TP / (TP + FP)
    recall    = TP + FN == 0 ? 0.0 : TP / (TP + FN)
    f1        = precision + recall == 0 ? 0.0 : 2 * (precision * recall) / (precision + recall)

    println("\nEvaluation Metrics for $filename")
    println("TP = $TP | FP = $FP | FN = $FN")
    println("Precision = $(round(precision * 100, digits=2))%")
    println("Recall    = $(round(recall * 100, digits=2))%")
    println("F1 Score  = $(round(f1 * 100, digits=2))%")

    return (;precision, recall, f1, TP, FP, FN)
end

compute_metrics (generic function with 2 methods)

### Saving Metrics

In [27]:
function save_model(model, filename::String)
    mkpath("models")
    model_path = joinpath("models", "model_" * replace(filename, "/" => "_") * ".bson")
    @save model_path model
end

function save_metrics(filename::String, metrics::NamedTuple)
    mkpath("results")
    results_file = joinpath("results", "unsupervsied_metrics.csv")

    header = ["filename", "TP", "FP", "FN", "Precision (%)", "Recall (%)", "F1 Score (%)"]

    row = [
        filename, metrics.TP, metrics.FP, metrics.FN,
        round(metrics.precision * 100, digits=2),
        round(metrics.recall * 100, digits=2),
        round(metrics.f1 * 100, digits=2)
    ]

    if isfile(results_file)
        open(results_file, "a") do io
            writedlm(io, [row], ',')
        end
    else
        open(results_file, "w") do io
            writedlm(io, [header], ',')
            writedlm(io, [row], ',')
        end
    end
end

save_metrics (generic function with 1 method)

### Implementation

In [30]:
data_root = "data"
label_path = "labels/combined_windows.json"
param_path = "params_unsupervised.json"

params = load_params(param_path)

all_files = sort(collect(keys(params)))

for filename in all_files
    println("\nProcessing: $filename")

    file_path = joinpath(data_root, filename)

    # Load data and parameters
    ts, values = load_csv_data(file_path)
    config = params[filename]
    probation_ratio = config["probation_ratio"]
    calib_ratio = config["calib_ratio"]
    window_size = Int(config["window_size"])
    epochs = Int(config["epochs"])
    lr = Float64(config["lr"])
    dropout = Float64(config["dropout"])
    conf = Float64(config["conf"])

    # Split and normalize data
    train_data, calib_data, eval_data = split_and_normalize(Vector{Float32}(values), probation_ratio, calib_ratio)

    # Create sequences
    x_train, y_train = create_sequences(train_data, window_size)
    x_calib, y_calib = create_sequences(calib_data, window_size)
    x_eval,  y_eval  = create_sequences(eval_data,  window_size)

    # Train model
    model = train_model(
        x_train, y_train;
        window_size=window_size,
        epochs=epochs,
        lr=lr,
        dropout=dropout
    )

    # Classification
    threshold, calib_residuals = conformal_classification(
        model, x_calib, y_calib; confidence=conf)

    if !isfinite(threshold)
        save_metrics(filename, (;TP=0, FP=0, FN=0, precision=0.0, recall=0.0, f1=0.0, mse=0.0, mae=0.0))
        continue
    end

    # Evaluate
    eval_results = evaluate_model(model, x_eval, y_eval, threshold)

    # True labels & metrics
    n = length(values)
    n_prob = Int(floor(probation_ratio * n))
    ts_eval = ts[n_prob + window_size + 1 : end]

    true_flags = load_true_labels(label_path, filename, ts_eval)
    metrics = compute_metrics(filename, eval_results.anomaly_flags, true_flags)

    # Save model & metrics
    # save_model(model, filename)
    save_metrics(filename, metrics)
end

println("\nAll files processed successfully")


Processing: artificialNoAnomaly/art_daily_no_noise.csv
Epoch 1 -- Loss = 1.2715
Epoch 2 -- Loss = 1.2593
Epoch 3 -- Loss = 1.2473
Epoch 4 -- Loss = 1.2359
Epoch 5 -- Loss = 1.2227
Epoch 6 -- Loss = 1.2225
Epoch 7 -- Loss = 1.2168
Epoch 8 -- Loss = 1.2021
Epoch 9 -- Loss = 1.2079
Epoch 10 -- Loss = 1.197
Epoch 11 -- Loss = 1.1886
Epoch 12 -- Loss = 1.1785
Epoch 13 -- Loss = 1.1714
Epoch 14 -- Loss = 1.168
Epoch 15 -- Loss = 1.1703
Epoch 16 -- Loss = 1.1514
Epoch 17 -- Loss = 1.1388
Epoch 18 -- Loss = 1.1433
Epoch 19 -- Loss = 1.1345
Epoch 20 -- Loss = 1.1253
Threshold = 0.7841

Evaluation Metrics for artificialNoAnomaly/art_daily_no_noise.csv
TP = 0 | FP = 1329 | FN = 0
Precision = 0.0%
Recall    = 0.0%
F1 Score  = 0.0%

Processing: artificialNoAnomaly/art_daily_perfect_square_wave.csv
Epoch 1 -- Loss = 0.9104
Epoch 2 -- Loss = 0.9014
Epoch 3 -- Loss = 0.8976
Epoch 4 -- Loss = 0.8914
Epoch 5 -- Loss = 0.8934
Epoch 6 -- Loss = 0.8788
Epoch 7 -- Loss = 0.8852
Epoch 8 -- Loss = 0.8797
Epo

# Supervised

### Parameter Loading

In [11]:
function load_params(json_path::String)::Dict{String, Dict{String, Float64}}
    if isfile(json_path)
        return JSON.parsefile(json_path)
    else
        error("params_supervised.json not found at $json_path")
    end
end

load_params (generic function with 1 method)

### Data Loading

In [12]:
function load_data_with_labels(filepath::String, labelpath::String)
    label_data = JSON.parsefile(labelpath)
    dataset_key = joinpath(splitpath(filepath)[end-1:end]...)

    df = CSV.read(filepath, DataFrame)
    timestamps = DateTime.(df.timestamp, dateformat"yyyy-MM-dd HH:MM:SS")
    values = Float32.(df.value)

    anomaly_windows = get(label_data, dataset_key, [])
    datetime_format = "yyyy-MM-dd HH:MM:SS.ssss"

    labels = Vector{Int}(undef, length(timestamps))
    for (i, ts) in enumerate(timestamps)
        labels[i] = any(DateTime(w[1], datetime_format) <= ts <=
        DateTime(w[2], datetime_format) for w in anomaly_windows) ? 1 : 0
    end

    return timestamps, values, labels
end

load_data_with_labels (generic function with 1 method)

### Data Normalisation and Splitting

In [13]:
function split_and_normalize(values, labels,
    train_ratio::Float64, calib_ratio::Float64)

    n = length(values)
    train_end = floor(Int, n * train_ratio)
    calib_end = floor(Int, n * (train_ratio + calib_ratio))

    train_raw = values[1:train_end]
    mean_train = mean(train_raw)
    std_train = std(train_raw)
    normal_values = (values .- mean_train) ./ std_train

    train_values = normal_values[1:train_end]
    train_labels = labels[1:train_end]

    calib_values = normal_values[train_end+1:calib_end]
    calib_labels = labels[train_end+1:calib_end]

    test_values = normal_values[calib_end+1:end]
    test_labels = labels[calib_end+1:end]

    return (;train_values, train_labels,
           calib_values, calib_labels,
           test_values, test_labels)
end

split_and_normalize (generic function with 2 methods)

### Data Sequencing

In [14]:
function create_sequences(values::Vector{Float32}, labels::Vector{Int}, window_size::Int)
    n_seq = length(values) - window_size
    x = Array{Float32}(undef, window_size, 1, n_seq)
    y = Array{Float32}(undef, 1, n_seq)

    for i in 1:n_seq
        x[:, 1, i] = values[i:(i + window_size - 1)]
        y[1, i] = labels[i + window_size - 1]
    end

    return x, y
end

create_sequences (generic function with 2 methods)

### Model Training

In [15]:
function build_model(window_size::Int, dropout::Float64)
    return Chain(
        LSTM(window_size => 64),
        Dropout(dropout),
        Dense(64 => 1),
        sigmoid
    )
end

function train_model(x_train::Array{Float32, 3}, y_train::Array{Float32,2};
    window_size::Int, epochs::Int, lr::Float64, dropout::Float64)

    model = build_lstm_model(window_size, dropout)
    opt_state = Flux.setup(ADAMW(lr), model)

    for epoch in 1:epochs
        Flux.reset!(model)

        loss, grads = Flux.withgradient(model) do m
            y_pred = m(x_train)
            y_true = reshape(y_train, size(y_pred))
            Flux.logitbinarycrossentropy(y_pred, y_true)
        end

        Flux.update!(opt_state, model, grads[1])
        println("Epoch $epoch -- Loss: $(round(loss, digits=4))")
    end

    return model
end

train_model (generic function with 1 method)

### Conformal Classification

In [16]:
function conformal_threshold(model, x_calib, y_calib, x_test, confidence::Float64)
    Flux.reset!(model)
    calib_prob = model(x_calib)
    residuals = vec(abs.(calib_prob .- y_calib))

    if any(isnan, residuals)
        return NaN, NaN, NaN
    end

    threshold = quantile(residuals, confidence)

    println("\nThreshold = $(round(threshold, digits=4))")

    Flux.reset!(model)
    test_prob = model(x_test)
    test_residuals = abs.(test_prob .- 0.0)
    predicted_anomalies = test_residuals .> threshold

    return predicted_anomalies, test_prob, threshold

end

conformal_threshold (generic function with 1 method)

### Evaluation

In [17]:
function compute_metrics(filename:: String, y_true::Vector{Int}, y_pred::BitVector)
    TP = sum((y_true .== 1) .& (y_pred .== true))
    FP = sum((y_true .== 0) .& (y_pred .== true))
    FN = sum((y_true .== 1) .& (y_pred .== false))

    precision = TP + FP == 0 ? 0.0 : TP / (TP + FP)
    recall = TP + FN == 0 ? 0.0 : TP / (TP + FN)
    f1 = precision + recall == 0 ? 0.0 : 2 * (precision * recall) / (precision + recall)

    println("\nEvaluation Metrics for $filename")
    println("TP = $TP | FP = $FP | FN = $FN")
    println("Precision: $(round(precision * 100, digits=2))%")
    println("Recall:    $(round(recall * 100, digits=2))%")
    println("F1 Score:  $(round(f1 * 100, digits=2))%")

    return (;precision, recall, f1, TP, FP, FN)
end

compute_metrics (generic function with 2 methods)

### Saving Metrics

In [18]:
function save_metrics(filename::String, metrics::NamedTuple)
    mkpath("results")
    results_file = joinpath("results", "supervised_metrics.csv")

    header = ["filename", "TP", "FP", "FN", "Precision (%)", "Recall (%)", "F1 Score (%)"]

    row = [
        filename, metrics.TP, metrics.FP, metrics.FN,
        round(metrics.precision * 100, digits=2),
        round(metrics.recall * 100, digits=2),
        round(metrics.f1 * 100, digits=2)
    ]

    if isfile(results_file)
        open(results_file, "a") do io
            writedlm(io, [row], ',')
        end
    else
        open(results_file, "w") do io
            writedlm(io, [header], ',')
            writedlm(io, [row], ',')
        end
    end
end

save_metrics (generic function with 1 method)

### Implementation

In [19]:
data_root = "data"
label_path = "labels/combined_windows.json"
param_path = "params_supervised.json"

params = load_params(param_path)

all_files = sort(collect(keys(params)))

for filename in all_files
    println("\nProcessing: $filename")

    file_path = joinpath(data_root, filename)

    # Load data and parameters
    timestamps, values, labels = load_data_with_labels(file_path, label_path)
    config = params[filename]
    train_ratio = config["train_ratio"]
    calib_ratio = config["calib_ratio"]
    window_size = Int(config["window_size"])
    epochs = Int(config["epochs"])
    lr = Float64(config["lr"])
    dropout = Float64(config["dropout"])
    conf = Float64(config["conf"])

    # Split and normalize data
    clean_data = split_and_normalize(values, labels, train_ratio, calib_ratio)

    # Create sequences
    x_train, y_train = create_sequences(clean_data.train_values, clean_data.train_labels, window_size)
    x_calib, y_calib = create_sequences(clean_data.calib_values, clean_data.calib_labels, window_size)
    x_test, y_test = create_sequences(clean_data.test_values, clean_data.test_labels, window_size)

    # Train model
    model = train_model(
        x_train, y_train;
        window_size=window_size,
        epochs=epochs,
        lr=lr,
        dropout=dropout
    )

    # Classification
    y_pred_class, test_prob, threshold = conformal_threshold(
        model, x_calib, y_calib, x_test, conf)

    if !isfinite(threshold)
        save_metrics(filename, (;TP=0, FP=0, FN=0, precision=0.0, recall=0.0, f1=0.0, mse=0.0, mae=0.0))
        continue
    end

    # Evaluation
    y_true = Int.(vec(y_test))
    y_pred = vec(y_pred_class)
    metrics = compute_metrics(filename, y_true, y_pred)

    # Save metrics
    save_metrics(filename, metrics)
end
println("\nAll files processed successfully")


Processing: artificialNoAnomaly/art_daily_no_noise.csv
Epoch 1 -- Loss: 0.7144
Epoch 2 -- Loss: 0.7027
Epoch 3 -- Loss: 0.6904
Epoch 4 -- Loss: 0.6801
Epoch 5 -- Loss: 0.6677
Epoch 6 -- Loss: 0.6554
Epoch 7 -- Loss: 0.6445
Epoch 8 -- Loss: 0.6315
Epoch 9 -- Loss: 0.6213
Epoch 10 -- Loss: 0.6087
Epoch 11 -- Loss: 0.5985
Epoch 12 -- Loss: 0.5874
Epoch 13 -- Loss: 0.5767
Epoch 14 -- Loss: 0.5629
Epoch 15 -- Loss: 0.5531
Epoch 16 -- Loss: 0.5434
Epoch 17 -- Loss: 0.5324
Epoch 18 -- Loss: 0.5215
Epoch 19 -- Loss: 0.5082
Epoch 20 -- Loss: 0.4989
Epoch 21 -- Loss: 0.4847
Epoch 22 -- Loss: 0.4737
Epoch 23 -- Loss: 0.4628
Epoch 24 -- Loss: 0.4494
Epoch 25 -- Loss: 0.4389
Epoch 26 -- Loss: 0.4283
Epoch 27 -- Loss: 0.4185
Epoch 28 -- Loss: 0.4076
Epoch 29 -- Loss: 0.394
Epoch 30 -- Loss: 0.3844

Threshold = 1.0347

Evaluation Metrics for artificialNoAnomaly/art_daily_no_noise.csv
TP = 0 | FP = 300 | FN = 0
Precision: 0.0%
Recall:    0.0%
F1 Score:  0.0%

Processing: artificialNoAnomaly/art_daily